In [64]:
import pandas as pd
import numpy as np
import math as m
import random as r
from IPython.display import display, HTML
import sympy as sy
import matplotlib.pyplot as plt

def isNegative(num):
    return  num < 0


def biseccion(threshold, f, epsilon, numIterations, method):    
    finish = False
    fp = f.diff(x)
    fpp = fp.diff(x)
    while(not finish):
        values = pd.DataFrame(columns=('x0', 'x1', 'xp', 'f(x0)', 'f(x1)', 'f(xp)'))
        i = 0
        x0 = r.uniform(threshold[0],threshold[1])
        x1 = r.uniform(threshold[0],threshold[1])
        while(isNegative(fp.evalf(subs={x:x0})) == isNegative(fp.evalf(subs={x:x1}))):
            x0 = r.uniform(threshold[0],threshold[1])
            x1 = r.uniform(threshold[0],threshold[1])
        if(x1 < x0):
            aux = x1
            x1 = x0
            x0 = aux
        error = m.fabs(x0 - x1)
        while((i < numIterations) and (error > epsilon)):
            xp = x0 + (m.fabs(x0-x1)/2)
            f_x0 = fp.evalf(subs={x:x0})
            f_x1 = fp.evalf(subs={x:x1})
            biseccion = fp.evalf(subs={x:xp})
            values.loc[len(values)] = [x0, x1, xp, f.evalf(subs={x:x0}), f.evalf(subs={x:x1}), f.evalf(subs={x:xp})]
            if(biseccion == 0):
                return values
            elif(isNegative(f_x0) == isNegative(biseccion)):
                x0 = xp
            elif(isNegative(f_x1) == isNegative(biseccion)): 
                x1 = xp
            i = i + 1 
            error = m.fabs(x0 - x1)
        if((method == 0 and fpp.evalf(subs={x:xp})>0) or (method == 1 and fpp.evalf(subs={x:xp})<0)):
            finish = True
    return values 

def newtonRaphson(threshold, f, epsilon, numIterations, method):
    fp = f.diff(x)
    fpp = fp.diff(x)
    finish = False
    while(not finish):
        values = pd.DataFrame(columns=('x', 'new_x', 'f(x)', 'f(x_new_x)'))
        error = -1
        i = 0
        point = r.uniform(threshold[0],threshold[1])
        while((i < numIterations) and (error > epsilon or error == -1)):
            f_point = fp.evalf(subs={x:point})
            fp_point = fpp.evalf(subs={x:point})
            point2 = point - (f_point)/(fp_point)
            f_newPoint = fp.evalf(subs={x:point2})
            error = m.fabs(f_newPoint)
            values.loc[len(values)] = [point, point2, f.evalf(subs={x:point}), f.evalf(subs={x:point2})]
            point = point2
            i = i + 1
        if((method == 0 and fpp.evalf(subs={x:point})>0) or (method == 1 and fpp.evalf(subs={x:point})<0)):
            finish = True
    return values

def getValue(values, function):
    optimalValue = values.loc[len(values) -1]
    if(function == "biseccion"):
        return optimalValue.loc['xp']
    elif(function == "newtonRaphson"):
        return optimalValue.loc['new_x']
    raise Exception('metodo ' + function + ' no existe!')
    
    
def optimization(threshold, f, epsilon, numIterations, objective, function):
    if(function == "newtonRaphson"):
        values = newtonRaphson(threshold, f, epsilon, numIterations, objective)
    elif(function == "biseccion"):
        values = biseccion(threshold, f, epsilon, numIterations, objective)
    optimalVaule = getValue(values, function)
    
    
    return HTML(""""<h1 style="color:red;" align="center">""" + function + """</h1>
        <h2>Historial</h2>
        """ + values.to_html() + """
        <h2>Valor optimo</h2>
        <p>""" + str(optimalVaule))
        

In [71]:
from sympy.abc import x

"""Para probar llamar al método optimization

        Parametros:
        
        threshold: tupla con el limite minimo y maximo (en ese orden).
        f: funcion a optimizar.
        epsilon: parametro de parada.
        numIterations: numero maximo de iteraciones.
        objective: indica si se encontrara el minimo o el maximo de la función, usar 0 o 1 respectivamente.
        function: indica si se usara el metodo de Newton-Raphson o biseccion, para ello pasar un string con el nombre
        "newtonRaphson" o "biseccion" respectivamente.
        
    Nota: El nombre de la variable de la funcion debe ser x obligatoriamente.
    
    Nota2: Aunque el método recibe cualquier funcion, no hace validacion de que esta sea unimodal, por tanto
    devolvera el primer maximo o minimo que encuentre.
    
    Nota3: Llamar la función en este mismo script, si se hace en un nuevo script se debe de realizar el import 
    from sympy.abc import x.
    
    Nota4: Recuerde correr el script anterior previamente a este.
    
    Abajo se muestra un ejemplo del funcionamiento del metodo
        """

y = -(0.3 * (x** 4)) + ((x ** 3)) - (1.8 * (x ** 2)) + (4 * x) 

v = optimization((1,4), y, 0.0001, 50, 1, "biseccion")
v


,x0,x1,xp,f(x0),f(x1),f(xp)
0,1.841262,2.180337,2.010799,4.05680643644148,3.74963849841432,3.99100887879178
1,1.841262,2.010799,1.926031,4.05680643644148,3.99100887879178,4.04331872279705
2,1.841262,1.926031,1.883646,4.05680643644148,4.04331872279705,4.05461873490888
3,1.841262,1.883646,1.862454,4.05680643644148,4.05461873490888,4.05681581882203
4,1.841262,1.862454,1.851858,4.05680643644148,4.05681581882203,4.05708253962778
5,1.851858,1.862454,1.857156,4.05708253962778,4.05681581882203,4.05701757821477
6,1.851858,1.857156,1.854507,4.05708253962778,4.05701757821477,4.05706709019829
7,1.851858,1.854507,1.853183,4.05708253962778,4.05706709019829,4.05707906419287
8,1.851858,1.853183,1.852520,4.05708253962778,4.05707906419287,4.05708186316407
9,1.851858,1.852520,1.852189,4.05708253962778,4.05708186316407,4.05708246657616
